In [42]:
import numpy as np
# import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
import pickle
from sklearn import set_config

In [43]:
train_data = pd.read_csv('../../data/raw/train.csv',  encoding= 'unicode_escape')
y_train = train_data["Unusual"]                      #defining the labels
X_train = train_data.drop(["Unusual"], axis=1)
test_data = pd.read_csv('../../data/raw/test.csv',  encoding= 'unicode_escape')
y_test = test_data["Unusual"]                      #defining the labels
X_test = test_data.drop(["Unusual"], axis=1)

In [44]:
numeric_features = X_train.select_dtypes(exclude=['object']).columns.tolist()

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

In [45]:
# Define custom transformers
class CustomTransformer:
    def __init__(self, parameter):
        self.parameter = parameter
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # perform custom transformation
        return X

In [46]:
# Create a list of all possible categories across both datasets
categories = pd.concat([X_train, X_test])['CellName'].unique()
categorical_features = ['CellName']
categorical_transformer = Pipeline(
    steps=[
        #("encoder", OneHotEncoder(handle_unknown="ignore"))
        ('encoder', OneHotEncoder(categories=[categories]))
    ]
)

In [47]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [48]:
# fit the preprocessor on the data
preprocessor.fit(X_train)
print(preprocessor)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Unnamed: 0', 'PRBUsageUL', 'PRBUsageDL',
                                  'meanThr_DL', 'meanThr_UL', 'maxThr_DL',
                                  'maxThr_UL', 'meanUE_DL', 'meanUE_UL',
                                  'maxUE_DL', 'maxUE_UL']),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(categories=[array(['9ALTE', '4CLTE', '6ALTE', '7WLTE', '6ULTE', '1BLTE', '10BLTE',
       '6WLTE', '3CLTE', '5ALTE', '10CLTE', '1CLTE', '8CLTE', '7ALTE',
       '5BLTE', '10ALTE', '7BLTE', '5CLTE', '7VLTE', '3BLTE', '8ALTE',
       '4ALTE', '6CLTE', '7ULTE', '6BLTE', '3ALTE', '7CLTE'

In [49]:
with open('PreprocessingPipeline.pkl', 'wb') as f:
    # write the preprocessor object to the file
    pickle.dump(preprocessor, f)

TEST

In [50]:
set_config(display="diagram")
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['Unnamed: 0', 'PRBUsageUL', 'PRBUsageDL',
                                  'meanThr_DL', 'meanThr_UL', 'maxThr_DL',
                                  'maxThr_UL', 'meanUE_DL', 'meanUE_UL',
                                  'maxUE_DL', 'maxUE_UL']),
                                ('cat',
                                 Pipeline(steps=[('encoder',
                                                  OneHotEncoder(categories=[array(['9ALTE', '4CLTE', '6ALTE', '7WLTE', '6ULTE', '1BLTE', '10BLTE',
       '6WLTE', '3CLTE', '5ALTE', '10CLTE', '1CLTE', '8CLTE', '7ALTE',
       '5BLTE', '10ALTE', '7BLTE', '5CLTE', '7VLTE', '3BLTE', '8ALTE',
       '4ALTE', '6CLTE', '7ULTE', '6BLTE', '3ALTE', '7CLTE', '6VLTE',
       '1ALTE', '2ALTE', '9BLTE', '4BLTE', '8BLTE'], dtype=object)]))]),
                                 ['CellName'])])

In [52]:
#X_processed = preprocess(X_test)
#print(X_processed.shap)
#column_names = pipe.named_steps['preprocessor'].transformers_[1][1]\
#   .named_steps["encoder"].get_feature_names(categorical_features)
# Create new pandas dataframe with transformed data and column names
#df_transformed = pd.DataFrame(X_processed, columns=np.concatenate((column_names, numeric_features)))